In [48]:
from CircleNet.system import Simulation
from CircleNet.shape import circle,Timer,get_log_normal
from CircleNet.shanghai_platform import MatchingPlatform, Driver, Passenger
from CircleNet.shanghai_platform_extension import extract_result_data
import numpy as np
import time


#TODO
#put writting functions somewhere else
#put extraction functions somewhere else
#reorganise parameters etc in other file


In [49]:
#automatic functions


class Write_csv():
    def __init__(self,name,keys_to_save):
        self.file_name=name
        self.the_string="{"+"},{".join(keys_to_save)+"}\n"
        with open(self.file_name , "w") as file :
            file.write(",".join(keys_to_save)+"\n")#write the first line
            assert len(keys_to_save)>0, "nothing to save"
    def write(self,**values):
        with open(self.file_name , "a") as file :
            file.write(self.the_string.format(**values))
 


In [50]:


def save_agents(simu,*files_info):
    """every file in files_info is a dictionnary
        file[name] -> name of the output file (preceded by {id}_)
        file[selection_function] -> send true for all agents concerned by this file
        file[extract_info] -> function that extract information from an agent (put in a dictionnary)
        file[info_list] -> list of info to store"""
    for file in files_info:
        file["writter"]=open("data/{}_{}.csv".format(simu.id_sim,file["name"] ),"w")
        file["writter"].write(",".join(file["info_list"])+"\n")
        file["the_string"]="{"+"},{".join(file["info_list"])+"}\n"
    for agent in simu:
        for file in files_info:
            if file["selection_function"](agent):
                data=file["extract_info"](agent)
                file["writter"].write(file["the_string"].format(**data))
    for file in files_info:
        file["writter"].close()
    

In [51]:
#simulation parameters

#MOVING
N_driver_list= [2000]#,3000,4000]#,5000,6000,7000,8000,9000,10000]
N_passenger_list= [2000]#,3000,4000]#,5000,6000,7000,8000,9000,10000]





PARAMETERS={
    
    
"speed": 25, #km/h
"radius": 25, #km
"end": 3, #h


#in second :
#Drivers:
"first_watching_before_first_departure": 5 * 60,
"window_size_of_departure": 15 * 60,
"time_elasticity": 5 * 60,
"fuel_cost": 0.5,
"watching_repetition_average": 60, # -> random
"watching_repetition_variance": 10,
"time_perception_average": 1 / 300, # = 24 * 50/100 / 3600 #50% of average income, in second   -> random
"time_perception_variance": 1 / 3000, #10% percent

#Passenger
"publishing_advance": 20 * 60
    }

def benefits(origin,destination,network):
    """Shanghai pricing"""
    distance=network.travel_distance(origin,destination)
    if distance < 3000: #3n first km fixprice
        return 11#in RMB
    if distance < 20000:#until 20km at 1.5RMB / km
        return 6.5 + distance * 1.5 # 11 + (distance - 3) * 1.5
    return 8.5 + distance # 28.5 + (distance - 20) *1   # 28.5 = 3 + (20 - 3) * 1.5
PARAMETERS["benefits"]=benefits

In [52]:
#SIMULATION CREATOR
def create_simulation(speed,radius,end,first_watching_before_first_departure,window_size_of_departure,
                       time_elasticity,fuel_cost,watching_repetition_average,watching_repetition_variance,
                      time_perception_average,time_perception_variance,publishing_advance,benefits,N_driver,N_passenger):
    
    #transformation in m, s
    speed/=3.6
    radius*=1000
    end*=3600
    #random functions
    watching_repetition=get_log_normal(watching_repetition_average,watching_repetition_variance)
    time_perception=get_log_normal(time_perception_average,time_perception_variance)
    

    #agents generator
    def SimpleDriver(simulation):
        t=simulation.timer.random_time()
        O=simulation.network.position_generator()
        D=simulation.network.position_generator()
        w=(t+first_watching_before_first_departure,t+first_watching_before_first_departure+window_size_of_departure)
        A=w[1]+simulation.network.travel_time(O,D)+time_elasticity
        return Driver(first_watching_time=t,
                      repetition_time=watching_repetition(),
                      departure_window=w,
                      position=O,destination=D,
                      last_arrival_time=A,
                      fuel_cost=fuel_cost,
                      time_perception=time_perception())
    def SimplePassenger(simulation):
        t=simulation.timer.random_time()
        O=simulation.network.position_generator()
        D=simulation.network.position_generator()
        return Passenger(publishing_time=t,
                         last_departure_time=t+publishing_advance,
                         position=O,destination=D)
    
    N=circle(radius,speed)
    T=Timer(end)
    simu=Simulation(N,T)
    simu.matchingAlgo=MatchingPlatform(benefits,simu)
    for i in range(N_driver):
        simu.add(SimpleDriver(simu))
    for i in range(N_passenger):
        simu.add(SimplePassenger(simu))
    return simu

In [53]:
#definition of savers :

#general
to_save=['id_sim', 'nb_passenger','nb_driver','execution_time', 'nb_match','driver_efficiency', 'passenger_efficiency',  
         'average_waiting_time','average_vks', 'total_vks']
result_file=Write_csv("data/first_sims_after_opt.csv",to_save)

#passengers
def extract_passenger_info(passenger):
    m=0
    d=""
    for _,action in passenger.story:
        if action[0] is "matched":
            m=1
            d=action[1]["driver"]
            break
    out={"id_num":passenger.id_number,"departure_t":passenger.last_departure_time,"matched":m,"driver":d}
    out["Ox"],out["Oy"]=passenger.position
    out["Dx"],out["Dy"]=passenger.destination
    return out
file_passenger={"name":"passengers","selection_function":lambda x:isinstance(x,Passenger),
                "extract_info":extract_passenger_info,"info_list":["id_num","Ox","Oy","Dx","Dy","departure_t","matched","driver","waiting_time"]}

#drivers
def extract_driver_info(driver):
    m=0
    p,v,d="","",""
    for _,action in driver.story:
        if action[0] is "matched":
            m=1
            p=action[1]["passenger"]
            break
    out={"id_num":driver.id_number,"departure_t":driver.departure_window[1],"matched":m,"passenger":p,
         "value_of_time":driver.time_perception,"watching_repetition_time":driver.repetition_time,"trip_vks":v,"detour":d}
    out["Ox"],out["Oy"]=driver.position
    out["Dx"],out["Dy"]=driver.destination
    return out
file_driver={"name":"drivers","selection_function":lambda x:isinstance(x,Driver),
                "extract_info":extract_driver_info,
                "info_list":["id_num","Ox","Oy","Dx","Dy","departure_t","value_of_time","watching_repetition_time","matched","passenger","trip_vks","detour"]}



In [54]:
#execution

for N_driver in N_driver_list:
    for N_passenger in N_passenger_list :
        print("begining of simulation with ",N_driver," drivers and ",N_passenger," passengers")
        #construction
        simu=create_simulation(**PARAMETERS,N_driver=N_driver,N_passenger=N_passenger)
        #execution
        simu()
        results=extract_result_data(simu)
        #saving
        result_file.write(**results)
        save_agents(simu,file_passenger,file_driver)
    


begining of simulation with  2000  drivers and  2000  passengers
